In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
logger = logging.getLogger("noisepy.seis.io")
logger.setLevel("INFO")

In [ ]:
import os
import numpy as np
import h5py
import obspy
import matplotlib.pyplot as plt

from noisepy.seis.io.h5store import DASH5DataStore
from datetime import datetime, timezone, timedelta
from datetimerange import DateTimeRange
from noisepy.seis.io.datatypes import CCMethod, ConfigParameters, FreqNorm, RmResp, StackMethod, TimeNorm  
from noisepy.seis.io.utils import fs_join
from noisepy.seis.io.numpystore import NumpyCCStore, NumpyStackStore

from noisepy.seis import cross_correlate, stack_cross_correlations, __version__  

In [ ]:
DAS_DATA = "../../../SeaDAS-tomo/data/seadas/"

# timeframe for analysis
start = datetime(2022, 12, 31, 2, 0, 0, tzinfo=timezone.utc)
end = datetime(2022, 12, 31, 3, 0, 0, tzinfo=timezone.utc)
time_range = DateTimeRange(start, end)
print(time_range)

In [ ]:
raw_store = DASH5DataStore(path = DAS_DATA,
                           sampling_rate=100,
                           channel_numbers=list(np.arange(500, 800, 10)),
                           file_naming = "seadasn_%Y-%m-%d_%H-%M-%S_GMT.h5",
                           array_name = "SeaDAS",
                           date_range = time_range) # Store for reading raw data from S3 bucket
raw_store.fs

In [ ]:
span = raw_store.get_timespans()
channels = raw_store.get_channels(span[0])
d = raw_store.read_data(span[2], channels[0])
d.stream

In [ ]:
path = "./das_data" 

os.makedirs(path, exist_ok=True)
cc_data_path = os.path.join(path, "CCF")
stack_data_path = os.path.join(path, "STACK")

In [ ]:
config = ConfigParameters()
config.start_date = start
config.end_date = end
config.acorr_only = False # only perform auto-correlation or not
config.xcorr_only = True # only perform cross-correlation or not

config.inc_hours = 0
config.samp_freq= 25  # (int) Sampling rate in Hz of desired processing (it can be different than the data sampling rate)
config.cc_len= 60  # (float) basic unit of data length for fft (sec)
config.step= 60.0  # (float) overlapping between each cc_len (sec)

config.ncomp = 1  # 1 or 3 component data (needed to decide whether do rotation)

config.stationxml= False  # station.XML file used to remove instrument response for SAC/miniseed data
      # If True, the stationXML file is assumed to be provided.
config.rm_resp= RmResp.NO  # select 'no' to not remove response and use 'inv' if you use the stationXML,'spectrum',

############## NOISE PRE-PROCESSING ##################
config.freqmin,config.freqmax = 1., 10.0  # broad band filtering of the data before cross correlation
config.max_over_std  = 100  # threshold to remove window of bad signals: set it to 10*9 if prefer not to remove them

################### SPECTRAL NORMALIZATION ############
config.freq_norm= FreqNorm.RMA  # choose between "rma" for a soft whitening or "no" for no whitening. Pure whitening is not implemented correctly at this point.
config.smoothspect_N = 10  # moving window length to smooth spectrum amplitude (points)
    # here, choose smoothspect_N for the case of a strict whitening (e.g., phase_only)

#################### TEMPORAL NORMALIZATION ##########
config.time_norm = TimeNorm.ONE_BIT # 'no' for no normalization, or 'rma', 'one_bit' for normalization in time domain,
config.smooth_N= 100  # moving window length for time domain normalization if selected (points)

############ cross correlation ##############
config.cc_method= CCMethod.XCORR # 'xcorr' for pure cross correlation OR 'deconv' for deconvolution;
    # FOR "COHERENCY" PLEASE set freq_norm to "rma", time_norm to "no" and cc_method to "xcorr"

config.substack = False  # True = smaller stacks within the time chunk. False: it will stack over inc_hours
config.substack_len = config.cc_len  # how long to stack over (for monitoring purpose): need to be multiples of cc_len

config.maxlag= 10  # lags of cross-correlation to save (sec)

config.net_list = ["*"]

In [ ]:
cc_store = NumpyCCStore(cc_data_path) # Store for writing CC data
span = raw_store.get_timespans()
channel_list=raw_store.get_channels(span[0])
d = raw_store.read_data(span[0], channel_list[2])
d.stream

In [ ]:
# ⛔️ this is where there are too many logging that overflows the notebook limit

cross_correlate(raw_store, config, cc_store)

In [ ]:
# open a new cc store in read-only mode since we will be doing parallel access for stacking
cc_store = NumpyCCStore(cc_data_path, mode="r")
stack_store = NumpyStackStore(stack_data_path)
config.stack_method = StackMethod.LINEAR

In [ ]:
stack_cross_correlations(cc_store, stack_store, config)

In [ ]:
pairs = stack_store.get_station_pairs()
print(f"Found {len(pairs)} station pairs")
sta_stacks = stack_store.read_bulk(time_range, pairs) # no timestamp used in ASDFStackStore

In [ ]:
for i in sta_stacks:
    src = int(i[0][0].name)
    rec = int(i[0][1].name)
    if src == 500:
        dist = abs(src - rec)
        plt.plot(i[1][0].data + dist/300, color = 'k', alpha=0.5)
# plt.ylim([-0.05, 0.2])